# Experiments with mindelay

In [ ]:
# Loading functions for parking functions
load("parking_functions.py")

# Loading Giovanni's code for Dyck paths and mindelay

def menk(aa,k): #remove k from each element of a list
    return [a-k for a in aa]

def piuk(aa,k): #add k to each element of a list
    return [a+k for a in aa]

from itertools import product
from numpy import argmin

def easy_bounce(path): #calculate bounce stat for Dyck Paths ([1,1,-1,1,-1,-1]) from 0,0 to n,n
    pp=deepcopy(path)
    n=len(pp)
    gg=[sum(pp[:i]) for i in range(n)]
    c=-min(gg)
    j=argmin(gg)
    start=0
    pos=start
    bounce=[]
    i=0
    bb=0
    bs=[]
    i=0
    while i<n:
        if sum(bounce[:i])==0:
            bounce.append(1)
        else:
            if bounce[-1]==-1:
                bounce.append(-1)
            else:
                if sum(bounce[:i])==sum(pp[:i]):
                    if pp[i]==-1:
                        bb+=1
                    bounce.append(pp[i])
                else:
                    bounce.append(1)
        bs.append(bb)
        i+=1
    return sum([bs[i] for i in range(n) if pp[i]==1])

def path_ord(p1,p2): #It is true if path p1 is above path p2 (paths with 1 and -1)
    n=len(p1)
    g1=[sum(p1[:i]) for i in range(n)]
    g2=[sum(p2[:i]) for i in range(n)]
    x=True
    i=0
    while x and i<n:
        if g1[i]<g2[i]:
            x=False
        i+=1
    return x

def my_Paths(n): #list of all paths from 0,0 to n,n (with 1 and -1)
    return [list(seq) for seq in product([-1, 1], repeat=2*n) if sum(seq) == 0]

def my_DyckPaths(n): #list of all Dyck paths from 0,0 to n,n (with 1 and -1)
    dd=[(-1)**i for i in range(2*n)]
    return [pp for pp in my_Paths(n) if path_ord(pp,dd)]

def from_my_to_area(pp): #convert a path with 1 and -1 in its complete! area word
    aa=list(pp)
    n=len(aa)/2
    return [[sum(aa[:i]) for i in range(2*n) if aa[i]==1][j]+n-j for j in range(n)]

def from_area_to_my(pp): #convert a completh area word in a path with 1 and -1
    n=len(pp)
    bb=list(pp)+[0]
    x=[]
    for i in range(n):
        x=x+[1]+[-1 for a in range(bb[i]-bb[i+1])]
    return x

def sc_bo(n,k,dizi=[],perc=[],start=0): 
    #return a list of dictionaries, such that the i-th ([i-1]...) is the list of all paths in n,k*n rectangular
    #above the diagonal, with the bounce decomposition statistic
    if start==0:
        dizi=[{} for a in range(k+1)]    
        dizi[1].update({tuple(menk(from_my_to_area(a),1)):easy_bounce(a) for a in my_DyckPaths(n)})
        perc=[{}]+[{tuple(menk(a,i)):1 for a in kDyckPaths(n,i)} for i in range(1,k+1)]
    if len(dizi[k])>0:
        return [dizi[j] for j in range(1,k+1)]
    km1=sc_bo(n,k-1,dizi,perc,1)[-1]
    for aa in km1:
        for bb in dizi[1]:
            cc=[list(aa)[i]+list(bb)[i] for i in range(n)]
            if tuple(cc) in perc[k]:
                new=easy_bounce(from_area_to_my(list(bb)))+dizi[k-1][aa]
                if tuple(cc) in dizi[k]:
                    if dizi[k][tuple(cc)]>new:
                        dizi[k][tuple(cc)]=new
                else:
                    dizi[k].update({tuple(cc):new})
    return [dizi[j] for j in range(1,k+1)]
        


# Converting dyck paths to corresponding parking functions
def dyck_to_pf(n,k,path):
    return ParkFunc(n,k,w_area=path,w_label=[i+1 for i in range(n)])

Now we want to try and find out why pmaj and mindelay are the same...

In [14]:
sc_bo_info(4,3)

{(3, 2, 1, 0): [6, [[4, 3, 2, 1]]], (3, 2, 1, 1): [5, [[4, 3, 2, 2]]], (3, 2, 2, 0): [4, [[4, 3, 3, 1]]], (3, 2, 2, 1): [4, [[4, 3, 3, 2]]], (3, 2, 2, 2): [3, [[4, 3, 3, 3]]], (3, 3, 1, 0): [3, [[4, 4, 2, 1]]], (3, 3, 1, 1): [2, [[4, 4, 2, 2]]], (3, 3, 2, 0): [3, [[4, 4, 3, 1]]], (3, 3, 2, 1): [2, [[4, 4, 3, 2]]], (3, 3, 2, 2): [2, [[4, 4, 3, 3]]], (3, 3, 3, 0): [1, [[4, 4, 4, 1]]], (3, 3, 3, 1): [1, [[4, 4, 4, 2]]], (3, 3, 3, 2): [1, [[4, 4, 4, 3]]], (3, 3, 3, 3): [0, [[4, 4, 4, 4]]]}


TypeError: unsupported operand parent(s) for +: 'Integer Ring' and '<class 'list'>'

In [3]:
n=3
k=3
#3,12  4,8   5,8  6,5
diz=sc_bo(n,k)
print(diz)
for bb in diz[-1]:#check if the bounce decomposition statistic is the same of the new pmaj
    if diz[-1][bb]!=ParkFunc(n,n*k,w_area=piuk(bb,k),w_label=list(range(1,n+1))).pmaj():
        print(bb)
print('ciao')

[{(2, 1, 0): 3, (2, 1, 1): 2, (2, 2, 0): 1, (2, 2, 1): 1, (2, 2, 2): 0}, {(4, 2, 0): 6, (4, 2, 1): 5, (4, 3, 0): 4, (4, 3, 1): 3, (4, 3, 2): 3, (4, 2, 2): 4, (4, 3, 3): 2, (4, 4, 0): 2, (4, 4, 1): 2, (4, 4, 2): 1, (4, 4, 3): 1, (4, 4, 4): 0}, {(6, 3, 0): 9, (6, 3, 1): 8, (6, 4, 0): 7, (6, 4, 1): 6, (6, 4, 2): 5, (6, 3, 2): 7, (6, 4, 3): 5, (6, 5, 0): 5, (6, 5, 1): 4, (6, 5, 2): 4, (6, 5, 3): 3, (6, 5, 4): 3, (6, 3, 3): 6, (6, 4, 4): 4, (6, 5, 5): 2, (6, 6, 0): 3, (6, 6, 1): 3, (6, 6, 2): 2, (6, 6, 3): 2, (6, 6, 4): 1, (6, 6, 5): 1, (6, 6, 6): 0}]
ciao
